## Preprocessing

In [121]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import numpy as np
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [122]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [123]:
# Determine the number of unique values in each column.
unique_value_counts = application_df.nunique()
print(unique_value_counts)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [124]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [125]:
# Choose a cutoff value
cutoff_value = 500  # You can adjust this cutoff value as needed

# Get the value counts of application types
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

# Identify application types to be replaced with "Other"
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index

# Replace in the DataFrame
application_df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other", inplace=True)

# Check to make sure binning was successful
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [126]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [127]:
# You may find it helpful to look at CLASSIFICATION value counts >1

classification_counts=application_df['CLASSIFICATION'].value_counts().loc[lambda x : x >1]

In [128]:
# Choose a cutoff value (e.g., 1000) for classification counts
cutoff_value = 1000

# Get the counts of each classification
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Create a list of classifications to be replaced based on the cutoff value
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()


# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [129]:
# Convert categorical data to numeric with `pd.get_dummies`

df = pd.DataFrame(application_df)

# Convert 'Category' column to dummy variables
df = pd.get_dummies(df).astype(int)
print(df.nunique())

STATUS                             2
ASK_AMT                         8747
IS_SUCCESSFUL                      2
APPLICATION_TYPE_Other             2
APPLICATION_TYPE_T10               2
APPLICATION_TYPE_T19               2
APPLICATION_TYPE_T3                2
APPLICATION_TYPE_T4                2
APPLICATION_TYPE_T5                2
APPLICATION_TYPE_T6                2
APPLICATION_TYPE_T7                2
APPLICATION_TYPE_T8                2
AFFILIATION_CompanySponsored       2
AFFILIATION_Family/Parent          2
AFFILIATION_Independent            2
AFFILIATION_National               2
AFFILIATION_Other                  2
AFFILIATION_Regional               2
CLASSIFICATION_C1000               2
CLASSIFICATION_C1200               2
CLASSIFICATION_C2000               2
CLASSIFICATION_C2100               2
CLASSIFICATION_C3000               2
CLASSIFICATION_Other               2
USE_CASE_CommunityServ             2
USE_CASE_Heathcare                 2
USE_CASE_Other                     2
U

In [130]:
# Split our preprocessed data into our features and target acommand:cellOutput.enableScrolling?4725a3e3-bf82-48ab-a077-800a828a58c4rrays
# Assuming you have a DataFrame 'application_df' with preprocessed data

# Define your features (X) and target (y)
X = df.drop(columns=['IS_SUCCESSFUL'],axis=1)
y = df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=70)

In [131]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [132]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()
input_dim = X_train.shape[1]
# # First hidden layer 
nn_model.add(Dense(units=80, activation='relu', input_dim=input_dim))


# Second hidden layer 
nn_model.add(Dense(units=30, activation='relu'))

# Output layer
nn_model.add(Dense(units=1, activation='sigmoid'))  # For binary classification

# Check the structure of the model
nn_model.summary()


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 80)                3520      
                                                                 
 dense_23 (Dense)            (None, 30)                2430      
                                                                 
 dense_24 (Dense)            (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [133]:
# Compile the model
# Assuming you've defined your model as 'nn' (as per your previous code)

# Compile the model

nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [134]:
# Define the ModelCheckpoint callback
model_checkpoint = ModelCheckpoint(
    filepath='model_weights_epoch.h5',
    save_weights_only=True,
    period=5  # Save weights every 5 epochs
)

In [135]:
# Train your model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50,callbacks=model_checkpoint)

Epoch 1/50
804/804 [==============================] - 1s 649us/step - loss: 0.5711 - accuracy: 0.7214
Epoch 2/50
804/804 [==============================] - 1s 622us/step - loss: 0.5566 - accuracy: 0.7290
Epoch 3/50
804/804 [==============================] - 0s 612us/step - loss: 0.5530 - accuracy: 0.7306
Epoch 4/50
804/804 [==============================] - 1s 624us/step - loss: 0.5513 - accuracy: 0.7319
Epoch 5/50
804/804 [==============================] - 1s 653us/step - loss: 0.5491 - accuracy: 0.7335
Epoch 6/50
804/804 [==============================] - 1s 650us/step - loss: 0.5487 - accuracy: 0.7323
Epoch 7/50
804/804 [==============================] - 0s 614us/step - loss: 0.5478 - accuracy: 0.7332
Epoch 8/50
804/804 [==============================] - 1s 642us/step - loss: 0.5464 - accuracy: 0.7339
Epoch 9/50
804/804 [==============================] - 0s 617us/step - loss: 0.5468 - accuracy: 0.7348
Epoch 10/50
804/804 [==============================] - 1s 655us/step - loss: 0.546

In [136]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5534 - accuracy: 0.7259 - 169ms/epoch - 629us/step
Loss: 0.5533592104911804, Accuracy: 0.7259474992752075


In [137]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity.h5")

c:\Users\Sujatha\anaconda3\envs\dev\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
